# [Graph Search, Shortest Paths, and Data Structures - Week4](https://www.coursera.org/learn/algorithms-graphs-data-structures/home/week/4)

- Hashing
- Universal Hashing
- Bloom Filters

SUGGESTED READINGS FOR WEEK 4: Algorithms Illuminated (Part 2), Chapter 12.

## XIV. HASHING: THE BASICS (Week 4)

### [Hash Tables: Operations and Applications](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/b2Uee/hash-tables-operations-and-applications)

#### slide ????

### [Hash Tables: Implementation Details, Part I](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/Ob0K7/hash-tables-implementation-details-part-i)

### [Hash Tables: Implementation Details, Part II](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/1Y8t1/hash-tables-implementation-details-part-ii)

## XV. UNIVERSAL HASHING (Week 4)

## XVI. BLOOM FILTERS (Week 4)